# Demo 09 - Testing and Evaluating Models

In this notebook we explore how to evaluate regression models and perform cross-validation.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/cmps3160
# !git pull
%cd _demos

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from scipy.stats import zscore
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Loading and Understanding the Data

For this work we will use the same data from the previous demo: ["Generalized body composition prediction equation for men using simple measurement techniques"](http://staff.pubhealth.ku.dk/~tag/Teaching/share/data/Bodyfat.html)


In [ ]:
# Load the Penrose Data
# Select a subste of columns
# Z-score all columns
# Make pairplot
input_names = ['Age', 'Neck', 'Forearm', 'Wrist']
output_name = 'bodyfat'
df = pd.read_csv("./data/bodyfat.csv")[input_names + [output_name]].apply(zscore)
sns.pairplot(df)
df.head()

## Linear Regression
Scikit-learn implements [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) as well.

All regression/classification classes in scikit-learn assume two functions:

- `.fit(X, y)`: fit/train the model giving training data `X` (the feature matrix) and `y` (the true values)
- `.predict(X)`: given a feature matrix, return the predicted `y` values.

In [ ]:
# Let's set the X and y variables.
X = df[input_names]
y = df[output_name]
display(X.head())
display(y.head())

In [ ]:
# Create a new regression model
lr = LinearRegression()
# fit the model on all data
lr.fit(X,y)
# predict on all data
y_predicted = lr.predict(X)

## Mean Average Error

How good is this model? To start, let's use Mean Average Error for evaluation:

$$MAE = \frac{1}{n}\sum_i^n |y - y_{\text{predicted}}|$$

In [ ]:
def mae(y, y_predicted):
  return (y - y_predicted).abs().mean()

mae(y, y_predicted)

### Interpreting MAE

Since we have already z-scored the $y$ variables, MAE$=1$ means that, on average, our prediction is one standard-deviation from the truth. That would not be very good for most applications.

We can also make a scatter plot comparing the true values with predicted values.

**Question:** What does the set of "perfect predictions" look like on this graph?

**Question:** What type of error is this? What did we do "wrong" from last lecture?

In [ ]:
# plot truth vs predicted
def plot_predictions(y, y_predicted):
  plt.figure(figsize=(6,5))
  plt.scatter(y_predicted, y)
  plt.xlabel('predicted y')
  plt.ylabel('true y')
  plt.title('MAE=%.3f' % mae(y, y_predicted))
  # make x/y ranges the same
  min_val = min(min(y), min(y_predicted))-.2
  max_val = max(max(y), max(y_predicted))+.2
  plt.xlim(min_val, max_val)
  plt.ylim(min_val, max_val)
  # plot line for "perfect predictions"
  plt.plot([min_val, max_val], [min_val, max_val], 'k-', lw=1, alpha=.5)
  plt.show() 

plot_predictions(y, y_predicted)

## Train/Test Splits

The MAE above is calculated on the training data. As we discussed, for prediction purposes, we don't really care much about the error on the training data. Instead, we care what the expected error will be on **new, unseen** data. This is what we call **generalization error**.

Splitting the data into a train and test set is a simple way to simulate what our error would be on a new, unseen sample of data.

Below, we'll split our data randomly into equal-sized train/test sets, then calculate error.

In [ ]:
def split_data(X, y, training_fraction=.5):
  # sample the training and testing indices 
  # setting random_state fixes the seed of the random number generator so we
  # get the same split each time (otherwise it can be very hard to reproduce/debug results!)
  train_idx = X.sample(frac=training_fraction, random_state=42).index
  test_idx = X.index[~X.index.isin(train_idx)]
  return (X.iloc[train_idx], y.iloc[train_idx], X.iloc[test_idx], y.iloc[test_idx])

X_train, y_train, X_test, y_test = split_data(X, y, training_fraction=0.5)
print('%d training and %d testing samples' % (len(X_train), len(X_test)))
print('first training instance: ')
display(X_train.head(1)) # this will change if I change random_state

### Test error

Now, we can train on the training data and test on the testing data.

In [ ]:
lr.fit(X_train,y_train)
y_predicted = lr.predict(X_test)
plot_predictions(y_test, y_predicted)

**Question: How does this error compare to the training error above?**

**Question:** This only gives us one measure of error? What if we got lucky or unlucky? How could we get more data for testing?



## Cross-validation
But, this is just one split of the data. Maybe we got lucky/unlucky? 

If we really care about estimating generalization error, we can repeat this splitting process many times to get a more robust and reliable estimate. The process we use is **cross-validation**.

![](https://github.com/nmattei/cmps3160/blob/master/_labs/images/k-folds.png?raw=1)


We implement cross-validation ourselves below, but you can also just use the [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html?highlight=kfold#sklearn.model_selection.KFold) implementation in sklearn.

In [ ]:
def cross_validation(X, y, n_folds=5):
  """
  Return an iterator over (training index, testing index) pairs
  for each cross-validation fold.
  """
  # test size -- we might lose a few at the end here if there is a remainder.
  test_size = int(len(y)/n_folds) 
  # all possible indices
  all_idx = set(np.arange(len(y)))
  for i in range(n_folds):
    # slides the test window forward
    test_idx = np.arange(i*test_size, i*test_size+test_size)
    # use the rest as training indices.
    train_idx = np.array(list(all_idx - set(test_idx)))
    # yield vs return: this makes the method Iterable
    yield (train_idx, test_idx)

# let's test on a simple example of 5-fold cross validation on 10 instances
X_small = X.head(10)  # what happens if we use 11 instances?
y_small = y.head(10)
# because we use yield, we can iterate over the output
# of the cross-validation function.
for train_idx, test_idx in cross_validation(X_small, y_small):
  print('train_idx=%s  test_idx=%s' % ((train_idx, test_idx)))

In [ ]:
# Now, let's compute the MAE for 10 folds of cross-validation on the full dataset.
maes = []
for train_idx, test_idx in cross_validation(X, y, n_folds=10):
  lr.fit(X.iloc[train_idx], y.iloc[train_idx])
  maes.append(mae(y.iloc[test_idx], lr.predict(X.iloc[test_idx])))

plt.figure()
plt.plot(maes, 'o')
plt.xlabel('fold number')
plt.ylabel('MAE')
plt.show()

display(maes)
print('mean MAE=%.3f, std=%.3f' % (np.mean(maes), np.std(maes)))

That's a lot of variation!!

This is not too surprising. Since we have less than 300 examples, changing the training set by 30 instances in each fold is more than a 10% change. Thus, we get very different models and very different test errors.

## Baselines

So, how do we know if the MAE above is any good? To put such numbers in context, it is often helpful to have a simple baseline to compare to. What is a simple baseline for this task?

<br><br><br><br>

Well, we could just return a random number sampled uniformly from the range of $y$ values seen in the training set.

Is there something slightly better that is also easy?

In [ ]:
# First, let's make a function to compute the mean error and stddev
def cv_error(X, y, model):
  maes = []
  for train_idx, test_idx in cross_validation(X, y, n_folds=10):
    model.fit(X.iloc[train_idx], y.iloc[train_idx])
    maes.append(mae(y.iloc[test_idx], model.predict(X.iloc[test_idx])))  
  return np.mean(maes), np.std(maes)

lr_mae, lr_std = cv_error(X, y, lr)
print('linear regression: mae=%.3f std=%.3f' % (lr_mae, lr_std))

In [ ]:
class RandomBaseline:
  """
  A random regression baseline that just returns a number
  selected uniformly within the range of the max/min values
  seen in the training data.
  """
  def __init__(self):
    # set our seed for reproducibility
    np.random.seed(42)

  def fit(self, X, y):
    self.min_y = min(y)
    self.max_y = max(y)

  def predict(self, X):
    return np.random.uniform(self.min_y, self.max_y, len(X))

rb = RandomBaseline()
rb.fit(X.head(), y.head())
rb.predict(X.head())

In [ ]:
class MeanBaseLine:
  """
  A slightly better baseline that returns the mean of the training set
  """
  def __init__(self):
    pass

  def fit(self, X, y):
    self.mean = np.mean(y)

  def predict(self, X):
    return [self.mean]*len(X)

me = MeanBaseLine()
me.fit(X.head(), y.head())
me.predict(X.head())

In [ ]:
rb_mae, rb_std = cv_error(X, y, RandomBaseline())
me_mae, me_std = cv_error(X, y, MeanBaseLine())
print('random baseline: mae=%.3f std=%.3f' % (rb_mae, rb_std))
print('mean baseline: mae=%.3f std=%.3f' % (me_mae, me_std))

**Whew, at least we're doing better than random**

Let's make a convenience function to plot the errors for a list of models.

In [ ]:
def compare_errors(errs, stds, labels):
  plt.figure()
  xs = np.arange(len(errs))
  plt.errorbar(xs, errs, fmt='bo', yerr=stds)
  plt.xticks(xs, labels)
  plt.ylabel('MAE')
  plt.show()
compare_errors([lr_mae, me_mae, rb_mae], [lr_std, me_std, rb_std], ['linear_regression', 'mean', 'random'])

## Polynomial Regression

To implement polynomial regression, we'll simply add new feature columns corresponding to higher-order transformations of the input features.

We'll implement this ourselves, though
sklearn also has a class for this: [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial)

In [ ]:
def make_polynomial(X, order=2):
  """
  Return a new DataFrame with higher-order transformations
  of each column
  """
  X_new = X.copy()
  for c in X.columns:
    for o in range(2, order+1):
      X_new['%s_%d' % (c, o)] = X[c]**o
  return X_new

display(X.head())
X_2 = make_polynomial(X, order=2)
X_2.head()

In [ ]:
from sklearn.linear_model import Ridge
lr_2_mae, lr_2_std = cv_error(X_2, y, lr)
print('quadratic regression: mae=%.3f std=%.3f' % (lr_2_mae, lr_2_std))

It doesn't look like adding these features helped at all.

This is large part due to the small training set. We already have pretty good model capacity with the linear model.

In [ ]:
# How about training error?
mae(lr.fit(X_2, y).predict(X_2), y)

In [ ]:
# In linear case, it was...
mae(lr.fit(X, y).predict(X), y)

## Overfitting

Let's see how badly we overfit as we consider higher order polynomials.

In [ ]:
def compare_polynomials(X, y, lr):
  train_errs = []
  test_errs = []
  stds = []
  labels = []
  for order in range(1,6):
    X_i = make_polynomial(X, order=order)
    train_errs.append(mae(lr.fit(X_i, y).predict(X_i), y))
    lr_i_mae, lr_i_std = cv_error(X_i, y, lr)
    test_errs.append(lr_i_mae)
    stds.append(lr_i_std)
    labels.append('order=%d' % order)
  return train_errs, test_errs, stds, labels


def plot_overfitting(train_errs, test_errs, stds, labels):
  plt.figure()
  xs = np.arange(len(test_errs))
  plt.errorbar(xs, test_errs, fmt='ro-', yerr=stds, label='Test MAE')
  plt.plot(xs, train_errs, 'bo-', label='Train MAE')
  plt.xticks(xs, labels)
  plt.ylabel('MAE')
  plt.legend(loc='upper left')
  plt.show()

train_errs, test_errs, stds, labels = compare_polynomials(X, y, lr)
plot_overfitting(train_errs, test_errs, stds, labels)
test_errs

(why does std increase with order??)

**So, which model should we choose here??**


<br><br><br>

This is a disappointing but not uncommon result -- we tried something more complex, and our generalization error did not improve. 

A big part of machine learning is figuring out new methods that are complex but avoid overfitting.

A simple approach is called **L2-regularization**.
This adds a penalty term that encourages smaller parameter values.

$$\beta^* \leftarrow \text{argmin}_\beta ~~~ \text{RSS}(\beta, X, y) + \alpha \sum_k \beta_k^2$$

Here, the RSS error function from before is augmented with the sum of the squares of each parameter. The bigger the $\beta$s become, the higher our error. 

The $\alpha$ parameter trades off the two terms in the objective --- we want to minimize RSS while minimizing weight magnitudes.

This approach is implemented in sklearn as [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [ ]:
from sklearn.linear_model import Ridge
train_errs, test_errs, stds, labels = compare_polynomials(X, y, Ridge(alpha=10))
plot_overfitting(train_errs, test_errs, stds, labels)
test_errs

We can get a slightly better error rate with Ridge. Though, the main advantage is that the variation in error is greatly reduced.